In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

反馈或问题？
如有任何反馈或问题，请打开一个问题。

# Python的Vertex SDK：使用未管理的图像数据集进行自定义训练示例

要使用这个Colaboratory笔记本，您需要将笔记本复制到您自己的谷歌云端硬盘中，并使用Colaboratory（或Colab）打开它。您可以运行每个步骤或单元，并查看其结果。要运行一个单元，使用Shift+Enter。Colab会自动显示每个单元中最后一行的返回值。要获取有关在Colab中运行笔记本的更多信息，请参阅[Colab欢迎页面](https://colab.research.google.com/notebooks/welcome.ipynb)。

这个笔记本演示了如何基于图像数据集创建一个自定义模型。您需要提供一个存储数据集的存储桶。

注意：在测试此SDK时，您可能要支付用于训练、预测、储存或使用其他GCP产品的费用。

# 安装用于Python的Vertex SDK，进行身份验证，然后将数据集上传到您的GCS存储桶

安装完SDK后，内核将会自动重新启动。您可能会看到这个错误信息“您的会话因为未知原因而崩溃”，这是正常的。

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 输入您的项目和GCS存储桶

在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
MY_PROJECT = "YOUR PROJECT"
MY_STAGING_BUCKET = "gs://YOUR BUCKET"  # bucket should be in same region as ucaip

初始化Vertex SDK的Python版

为Vertex AI初始化*client*

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=MY_PROJECT, staging_bucket=MY_STAGING_BUCKET)

# 编写您的训练脚本
- 将此单元格写入作为一个文件，该文件将用于自定义训练。
- 通过'run'函数的'args'参数传入一个Tensorflow Dataset URI，而不是使用托管的数据集。脚本将在训练时从URI下载数据。

In [ ]:
%%writefile training_script.py

# Source: https://cloud.google.com/vertex-ai/docs/tutorials/image-recognition-custom

import argparse
import logging
import os

import tensorflow as tf
import tensorflow_datasets as tfds

IMG_WIDTH = 128

def normalize_img(image):
    """Normalizes image.

    * Resizes image to IMG_WIDTH x IMG_WIDTH pixels
    * Casts values from `uint8` to `float32`
    * Scales values from [0, 255] to [0, 1]

    Returns:
      A tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color channels)
    """
    image = tf.image.resize_with_pad(image, IMG_WIDTH, IMG_WIDTH)
    return image / 255.


def normalize_img_and_label(image, label):
    """Normalizes image and label.

    * Performs normalize_img on image
    * Passes through label unchanged

    Returns:
      Tuple (image, label) where
      * image is a tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color
        channels)
      * label is an unchanged integer [0, 4] representing flower type
    """
    return normalize_img(image), label

def get_args():
  """Argument parser.
  Returns:
    Dictionary of arguments.
  """
  parser = argparse.ArgumentParser(description='Flower classification sample')
  parser.add_argument(
      '--tfds',
      default=None,
      help='The tfds URI from https://www.tensorflow.org/datasets/ to load the data from')

  args = parser.parse_args()
  return args

# Training settings
args = get_args()

if 'AIP_MODEL_DIR' not in os.environ:
    raise KeyError(
        'The `AIP_MODEL_DIR` environment variable has not been' +
        'set. See https://cloud.google.com/vertex-ai/docs/tutorials/image-recognition-custom/training'
    )
output_directory = os.environ['AIP_MODEL_DIR']

logging.info('Loading and preprocessing data ...')
dataset = tfds.load(args.tfds,
                    split='train',
                    try_gcs=True,
                    shuffle_files=True,
                    as_supervised=True)
dataset = dataset.map(normalize_img_and_label,
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.cache()
dataset = dataset.shuffle(1000)
dataset = dataset.batch(128)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

logging.info('Creating and training model ...')
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,
                           3,
                           padding='same',
                           activation='relu',
                           input_shape=(IMG_WIDTH, IMG_WIDTH, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(5)  # 5 classes
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
model.fit(dataset, epochs=10)

logging.info(f'Exporting SavedModel to: {output_directory}')
# Add softmax layer for intepretability
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
probability_model.save(output_directory)

启动一个训练任务来创建一个模型

一旦我们定义了您的训练脚本，我们将创建一个模型。

In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name="train-flowers-dist-1-replica",
    script_path="training_script.py",
    container_uri="gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest",
    requirements=["gcsfs==0.7.1"],
    model_serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest",
)
model = job.run(
    args=["--tfds", "tf_flowers:3.*.*"],
    replica_count=1,
    model_display_name="flowers-model",
)

部署您的模型，然后等待模型部署完成后再进行预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

在终端上进行预测
要进行预测，您需要一些花卉图片。您可以下载一些花朵照片，或使用下面提供的图片。

In [ ]:
!gsutil -m cp -R gs://cloud-ml-data/img/flower_photos/daisy/14221848160_7f0a37c395.jpg .
!gsutil -m cp -R gs://cloud-ml-data/img/flower_photos/tulips/13289268363_b9337d751e.jpg .
!gsutil -m cp -R gs://cloud-ml-data/img/flower_photos/sunflowers/14623719696_1bb7970208_n.jpg .

In [ ]:
import numpy as np
from PIL import Image

daisy_floats = np.array(Image.open("14221848160_7f0a37c395.jpg"))

In [ ]:
small_image = np.array(Image.fromarray(np.uint8(daisy_floats)).resize((128, 128)))

In [ ]:
endpoint.predict(instances=[small_image.tolist()])